## Rules
1. Four coloured team
2. Each team must be covered by 1 person everyday
* On weekdays, one employee can cover 2 teams
* On weekends, employees can only be on one team, but can cover 2 teams if in gold
* One cannot do both Red and Blue on weekends
* An employee cannot work for 3 weekends in a row
* Different days of the week have score. More the score, less ideal to work on:

Saturday < Friday < Sunday < Tuesday = Thursday < Wednesday = Monday<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(5)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(4)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(3)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(2)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(2)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(1)
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(1)
       
* Design the algorithm to find the best schedule based on the lowest overall total
* Some employees will request days off, they should not be penaltized to have more work

In [1]:
import random
import calendar
import pandas as pd

#### Random year & month

In [2]:
year = random.randint(1970,2100)
month = random.randint(1,12)
print("Year :" , year," Month: ",month)

#no of days
start_day, no_of_days = calendar.monthrange(year, month)
start_day = (calendar.day_name[start_day])
print("No. of days: ", no_of_days)
print("Starting day of the week: ", start_day)

Year : 2031  Month:  9
No. of days:  30
Starting day of the week:  Monday


#### Create month table

In [75]:
def get_score(weekday_name):
#     print(weekday_name)
    day_score = []
    for name in weekday_name:
        if name == 'Saturday' :
            day_score.append(5)
        elif name == 'Friday':
            day_score.append(4)
        elif name == 'Sunday':
            day_score.append(3)
        elif name == 'Tuesday' or name =='Thursday' :
            day_score.append(2)
        else:
            day_score.append(1)
    return day_score
            

def create_table(start, end):
    df = pd.DataFrame({'Date': pd.date_range(start, end)})
    df['Day'] = df.Date.dt.weekday_name
    df['Score'] = get_score(df.Date.dt.weekday_name)
    df['Blue'] = 'NA'
    df['Red'] = 'NA'
    df['Silver'] = 'NA'
    df['Gold'] = 'NA'
    return df

In [76]:
start_date = str(year) + '-' + str(month) +'-01'
end_date = str(year) + '-' + str(month) +'-'+str(no_of_days)

df_teams = create_table(start_date, end_date)
df_teams

,Date,Day,Score,Blue,Red,Silver,Gold
0,2031-09-01,Monday,1,NA,NA,NA,NA
1,2031-09-02,Tuesday,2,NA,NA,NA,NA
2,2031-09-03,Wednesday,1,NA,NA,NA,NA
3,2031-09-04,Thursday,2,NA,NA,NA,NA
4,2031-09-05,Friday,4,NA,NA,NA,NA
5,2031-09-06,Saturday,5,NA,NA,NA,NA
6,2031-09-07,Sunday,3,NA,NA,NA,NA
7,2031-09-08,Monday,1,NA,NA,NA,NA
8,2031-09-09,Tuesday,2,NA,NA,NA,NA
9,2031-09-10,Wednesday,1,NA,NA,NA,NA


#### List of random no. of employees

In [88]:
no_of_empl = random.randint(10,20)
empl_list = []
for i in range(1,no_of_empl+1):
    if(i<10):
        empl_list.append('emp_0' + str(i))
    else:
        empl_list.append('emp_' + str(i))
print(empl_list)


['emp_01', 'emp_02', 'emp_03', 'emp_04', 'emp_05', 'emp_06', 'emp_07', 'emp_08', 'emp_09', 'emp_10', 'emp_11', 'emp_12', 'emp_13', 'emp_14']


#### >> Random Holidays for Employees (TO BE DONE)

#### Create table with Employees

In [89]:
def employee_schedule(start, end, empl_list):
    df = pd.DataFrame({'Date': pd.date_range(start, end)})
    for emp in empl_list:
        df[emp] = 0
    return df
    
df_emp = employee_schedule(start_date, end_date, empl_list)
df_emp

,Date,emp_01,emp_02,emp_03,emp_04,emp_05,emp_06,emp_07,emp_08,emp_09,emp_10,emp_11,emp_12,emp_13,emp_14
0,2031-09-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2031-09-02,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2031-09-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2031-09-04,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2031-09-05,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2031-09-06,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,2031-09-07,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2031-09-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2031-09-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,2031-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### >> Handle few Employees who cannot be assigned weekends (TO BE DONE)

In [90]:
df_teams_temp = df_teams
for loc,date in enumerate(df_teams_temp['Date']):
#     print(date)
#     print(loc)
    #weekend
    if(date.weekday_name == 'Saturday' or date.weekday_name == 'Sunday'):
#         print('weekend')
        pass
        
    #Weekday
    else:
        #Get random choice for Gold
        gold_choice = random.choice(empl_list)
        df_teams_temp.at[loc, 'Gold'] = gold_choice
        print(gold_choice)
        
        #assign same to one of the other three team (red,blue,silver)
        with_gold = random.choice(['Red','Blue','Silver'])
        df_teams_temp.at[loc, with_gold] = gold_choice
        
        #assign different employee to remaining 2 team
        other_choice = random.choice([x for x in empl_list if x != gold_choice])
        if with_gold == 'Blue':
            df_teams_temp.at[loc, 'Red'] = other_choice
            df_teams_temp.at[loc, 'Silver'] = other_choice
        
        if with_gold == 'Red':
            df_teams_temp.at[loc, 'Blue'] = other_choice
            df_teams_temp.at[loc, 'Silver'] = other_choice
            
        if with_gold == 'Silver':
            df_teams_temp.at[loc, 'Blue'] = other_choice
            df_teams_temp.at[loc, 'Red'] = other_choice
        
        
df_teams_temp

emp_10
emp_01
emp_08
emp_14
emp_06
emp_07
emp_08
emp_09
emp_07
emp_12
emp_13
emp_03
emp_12
emp_04
emp_08
emp_01
emp_05
emp_12
emp_08
emp_11
emp_12
emp_13


/anaconda3/envs/mytfenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead
  


,Date,Day,Score,Blue,Red,Silver,Gold
0,2031-09-01,Monday,1,emp_06,emp_10,emp_06,emp_10
1,2031-09-02,Tuesday,2,emp_11,emp_01,emp_11,emp_01
2,2031-09-03,Wednesday,1,emp_08,emp_11,emp_11,emp_08
3,2031-09-04,Thursday,2,emp_03,emp_03,emp_14,emp_14
4,2031-09-05,Friday,4,emp_05,emp_05,emp_06,emp_06
5,2031-09-06,Saturday,5,NA,NA,NA,NA
6,2031-09-07,Sunday,3,NA,NA,NA,NA
7,2031-09-08,Monday,1,emp_13,emp_13,emp_07,emp_07
8,2031-09-09,Tuesday,2,emp_04,emp_04,emp_08,emp_08
9,2031-09-10,Wednesday,1,emp_14,emp_14,emp_09,emp_09


In [65]:
df1 = pd.DataFrame([[0, 2, 3], [0, 4, 1], [10, 20, 30]],
...                   index=[4, 5, 6], columns=['A', 'B', 'C'])
df1

,A,B,C
4,0,2,3
5,0,4,1
6,10,20,30


In [70]:
df1.at[4,'C'] = 10
df1

,A,B,C
4,0,2,10
5,0,4,1
6,10,20,30
